# Preprocessing and feature engineering

## Introduction

### Goal

The purpose of this notebook is to transform the raw wine review data into a clean, feature-rich dataset ready for the modeling phase. This involves rigorously cleaning the data, standardizing the text, and engineering new features that could improve our model's performance.

### Steps

1. Train/test split: Partition the data at the very beginning to create a training set for development and a test set for unbiased evaluation, preventing any data leakage.
2. Data cleaning: Systematically handle duplicates and missing values using strategies informed by our EDA, such as imputing prices based on group medians.
3. Text preprocessing: Create a reusable pipeline to clean and standardize the wine descriptions by lowercasing, removing punctuation and stop words, and lemmatizing the text.
4. Feature engineering: Create new, potentially predictive features, including `review_length`, `readability_score`, and a custom `value_score` to capture quality relative to price.
5. Corpus Creation: Combine the processed text with key categorical features to create a final, unified `corpus` for each wine that our NLP models will use.

## 1. Setup and imports

Goal: Import necessary libraries and load the dataset.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import textstat

In [2]:
# Set display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 80)

## 2. Data acquisition

Goal: Load the raw dataset.

In [3]:
df = pd.read_csv('winemag-data-130k-v2.csv', index_col=0)

In [4]:
print(f"Dataset loaded with {df.shape[0]} rows and {df.shape[1]} columns.")

Dataset loaded with 129971 rows and 13 columns.


## 3. Train-test split

Goal: Split the data before any cleaning or preprocessing to prevent data leakage.

In [5]:
# Split the dataframe into training (80%) and testing (20%) sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
print(f"Training set shape: {train_df.shape}")
print(f"Testing set shape:  {test_df.shape}")

Training set shape: (103976, 13)
Testing set shape:  (25995, 13)


## 4. Data cleaning and preprocessing

Goal: Clean the data by handling missing values and duplicates.

### 4.1. Handle duplicates

In [7]:
# We will keep the first occurrence of any duplicate descriptions. This is done first
# to ensure our imputation calculations are based on unique data.

train_before = train_df.shape[0]
test_before = test_df.shape[0]

train_df.drop_duplicates(subset=['description'], keep='first', inplace=True)
test_df.drop_duplicates(subset=['description'], keep='first', inplace=True)

train_after = train_df.shape[0]
test_after = test_df.shape[0]

print(f"Train set: Dropped {train_before - train_after} duplicate descriptions. Shape changed from {train_before} to {train_after}.")
print(f"Test set:  Dropped {test_before - test_after} duplicate descriptions. Shape changed from {test_before} to {test_after}.")

Train set: Dropped 6409 duplicate descriptions. Shape changed from 103976 to 97567.
Test set:  Dropped 403 duplicate descriptions. Shape changed from 25995 to 25592.


### 4.2. Handle missing essential columns

In [8]:
# Drop rows where 'description' or 'title' are missing in both sets.

train_before_na = train_df.shape[0]
test_before_na = test_df.shape[0]

train_df.dropna(subset=['description', 'title'], inplace=True)
test_df.dropna(subset=['description', 'title'], inplace=True)

train_after_na = train_df.shape[0]
test_after_na = test_df.shape[0]

print(f"Train set: Dropped {train_before_na - train_after_na} rows with missing essentials. Shape changed from {train_before_na} to {train_after_na}.")
print(f"Test set:  Dropped {test_before_na - test_after_na} rows with missing essentials. Shape changed from {test_before_na} to {test_after_na}.")

Train set: Dropped 0 rows with missing essentials. Shape changed from 97567 to 97567.
Test set:  Dropped 0 rows with missing essentials. Shape changed from 25592 to 25592.


### 4.3. Impute missing prices

In [9]:
# Create the price imputation map using only the training data to avoid data leakage.
price_imputation_map = train_df.groupby(['country', 'variety'])['price'].median()

In [10]:
# Apply the map to the training set
train_lookup_keys = train_df[['country', 'variety']].apply(tuple, axis=1)
train_imputed_prices = train_lookup_keys.map(price_imputation_map)
train_df['price'] = train_df['price'].fillna(train_imputed_prices)

In [11]:
# Apply the same map to the testing set
test_lookup_keys = test_df[['country', 'variety']].apply(tuple, axis=1)
test_imputed_prices = test_lookup_keys.map(price_imputation_map)
test_df['price'] = test_df['price'].fillna(test_imputed_prices)

In [12]:
# If any prices are still missing (e.g., a new country/variety in test set), fill with global median from train set
global_median_price = train_df['price'].median()
train_df['price'] = train_df['price'].fillna(global_median_price)
test_df['price'] = test_df['price'].fillna(global_median_price)

### 4.4. Handle missing categorical features

In [13]:
# Create the 'is_region_missing' feature before filling NaNs
train_df['is_region_missing'] = train_df['region_1'].isnull()
test_df['is_region_missing'] = test_df['region_1'].isnull()

In [14]:
# List of categorical columns to fill
cols_to_fill = ['region_1', 'winery', 'designation', 'region_2', 'taster_name', 'taster_twitter_handle', 'country', 'province', 'variety']
for col in cols_to_fill:
    if col in train_df.columns:
        train_df[col] = train_df[col].fillna("Unknown")
        test_df[col] = test_df[col].fillna("Unknown")

### 4.5. Final check

In [15]:
print(f"Cleaned training set shape: {train_df.shape}")
print(f"Cleaned testing set shape:  {test_df.shape}")
print("\nMissing values after cleaning (Train):")
print(train_df.isnull().sum())

Cleaned training set shape: (97567, 14)
Cleaned testing set shape:  (25592, 14)

Missing values after cleaning (Train):
country                  0
description              0
designation              0
points                   0
price                    0
province                 0
region_1                 0
region_2                 0
taster_name              0
taster_twitter_handle    0
title                    0
variety                  0
winery                   0
is_region_missing        0
dtype: int64


## 5. Text preprocessing

Goal: Standardize the review text for NLP models.

In [16]:
# Initialize the lemmatizer and stop words list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [17]:
def preprocess_text(text):
    """
    Applies a series of text cleaning steps.
    """
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word.isalpha()]
    return " ".join(tokens)

In [18]:
# Apply the preprocessing function to the description column
# We'll create a new column to store the processed text
train_df['processed_description'] = train_df['description'].apply(preprocess_text)
test_df['processed_description'] = test_df['description'].apply(preprocess_text)

In [19]:
# Display a before-and-after example to verify
print("Original: ", train_df['description'].iloc[0])
print("Processed:", train_df['processed_description'].iloc[0])

Original:  Earthy leather and black pepper notes play against tart cherry and raspberry flavors in this light-bodied red blended with 8% Petite Sirah. Bright acidity supports the fruit.
Processed: earthy leather black pepper note play tart cherry raspberry flavor lightbodied red blended petite sirah bright acidity support fruit


## 6. Feature engineering

Goal: Create new features to enrich the data.

### 6.1. Create text-based features

In [20]:
train_df['review_length'] = train_df['description'].apply(lambda x: len(x.split()))
test_df['review_length'] = test_df['description'].apply(lambda x: len(x.split()))

In [21]:
train_df['readability_score'] = train_df['description'].apply(textstat.flesch_reading_ease)
test_df['readability_score'] = test_df['description'].apply(textstat.flesch_reading_ease)

### 6.2. Create "value score" feature

In [22]:
# Create price brackets
price_bins = [0, 20, 50, 100, 500, df['price'].max()]
price_labels = ['0-20', '21-50', '51-100', '101-500', '500+']

In [23]:
train_df['price_bracket'] = pd.cut(train_df['price'], bins=price_bins, labels=price_labels, right=False)
test_df['price_bracket'] = pd.cut(test_df['price'], bins=price_bins, labels=price_labels, right=False)

In [24]:
# Normalize points within each variety and price bracket group
# We calculate the normalization stats (min/max) only on the training set
normalization_stats = train_df.groupby(['variety', 'price_bracket'])['points'].agg(['min', 'max'])

/var/folders/kv/g0g8nw2j0jbbsk8g5j6dxdbr0000gp/T/ipykernel_14244/2168348663.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  normalization_stats = train_df.groupby(['variety', 'price_bracket'])['points'].agg(['min', 'max'])


In [25]:
def calculate_value_score(row, stats):
    try:
        group_stats = stats.loc[(row['variety'], row['price_bracket'])]
        min_pts, max_pts = group_stats['min'], group_stats['max']
        if max_pts == min_pts:
            return 0.5 # If all wines in group have same score, give neutral value
        else:
            return (row['points'] - min_pts) / (max_pts - min_pts)
    except (KeyError, ValueError):
        return 0.5 # Default for groups not seen in training data

In [26]:
train_df['value_score'] = train_df.apply(lambda row: calculate_value_score(row, normalization_stats), axis=1)
test_df['value_score'] = test_df.apply(lambda row: calculate_value_score(row, normalization_stats), axis=1)

### 6.3. Create final corpus for modeling

In [27]:
train_df['corpus'] = train_df['processed_description'] + " " + train_df['variety'] + " " + train_df['country']
test_df['corpus'] = test_df['processed_description'] + " " + test_df['variety'] + " " + test_df['country']

In [29]:
# Display the dataframe with new features
display(train_df.head())

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,is_region_missing,processed_description,review_length,readability_score,price_bracket,value_score,corpus
104748,US,Earthy leather and black pepper notes play against tart cherry and raspberry...,Unknown,88,23.0,California,Dry Creek Valley,Sonoma,Virginie Boone,@vboone,Hobo 2014 Zinfandel (Dry Creek Valley),Zinfandel,Hobo,False,earthy leather black pepper note play tart cherry raspberry flavor lightbodi...,27,55.265833,21-50,0.533333,earthy leather black pepper note play tart cherry raspberry flavor lightbodi...
101219,US,"This is a heavy, rich, dry Syrah. It shows forward flavors of blackberries, ...",Unknown,85,25.0,California,Sonoma County,Sonoma,Unknown,Unknown,Bluenose 2005 Syrah (Sonoma County),Syrah,Bluenose,False,heavy rich dry syrah show forward flavor blackberry cherry currant mocha spi...,30,81.065000,21-50,0.294118,heavy rich dry syrah show forward flavor blackberry cherry currant mocha spi...
82261,US,"The winery's top of the top, this is a viscous, concentrated expression of t...",Darius II,92,225.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Darioush 2012 Darius II Cabernet Sauvignon (Napa Valley),Cabernet Sauvignon,Darioush,False,winery top top viscous concentrated expression grape blended merlot fruit pr...,54,43.130000,101-500,0.529412,winery top top viscous concentrated expression grape blended merlot fruit pr...
36717,Austria,"Dark purple, juicy wine, with spice and pepper flavors from the Zweigelt. Th...",Koenigsegg Zwei. 1,84,12.0,Burgenland,Unknown,Unknown,Roger Voss,@vossroger,Schloss Halbturn 2006 Koenigsegg Zwei. 1 Zweigelt (Burgenland),Zweigelt,Schloss Halbturn,True,dark purple juicy wine spice pepper flavor zweigelt fruit come fresh blackbe...,30,78.245000,0-20,0.200000,dark purple juicy wine spice pepper flavor zweigelt fruit come fresh blackbe...
128031,France,"This 20-acre vineyard produces a wine dominated by Merlot, lending the wine ...",Unknown,89,23.0,Bordeaux,Moulis-en-Médoc,Unknown,Roger Voss,@vossroger,Château Guitignan 2011 Moulis-en-Médoc,Bordeaux-style Red Blend,Château Guitignan,False,vineyard produce wine dominated merlot lending wine smooth texture wine rich...,38,75.983596,21-50,0.473684,vineyard produce wine dominated merlot lending wine smooth texture wine rich...


## 7. Save processed data

Goal: Save the cleaned and feature-engineered dataframes for the next notebook.

In [30]:
train_df.to_csv('train_processed.csv', index=False)
test_df.to_csv('test_processed.csv', index=False)

## Conclusion and key insights

This notebook successfully executed the critical data preparation phase of our project. By splitting the data first and applying our cleaning steps consistently, we've created a robust foundation for model training and evaluation. The key outcome is the creation of two clean datasets, `train_processed.csv` and `test_processed.csv`, which are now free of missing values and enriched with several new features designed to capture more nuance than the text alone. The `value_score` and `readability_score` provide additional dimensions for our hybrid model, and the final `corpus` is a clean, standardized text representation of each wine. We are now fully prepared to move on to the next notebook, where we will use these processed datasets to experiment with and evaluate our different recommendation models.